In [1]:
import numpy as np
import trimesh
import tqdm 

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    # 1. compute area of each triangles
    area = np.zeros(faces.shape[0])
    v0 = vertices[faces[:, 0]]
    v1 = vertices[faces[:, 1]]
    v2 = vertices[faces[:, 2]]
    cross = np.cross(v1 - v0, v2 - v0)
    area = np.linalg.norm(cross, axis=1) / 2
    prob = area / np.sum(area)
    sampled_face_indices = np.random.choice(
        len(faces), 
        size=sample_num,
        p=prob,
        replace=True
    )
    u = np.random.rand(sample_num)
    v = np.random.rand(sample_num)
    mask = u + v > 1  # 处理超出三角形区域的情况
    u[mask] = 1 - u[mask]
    v[mask] = 1 - v[mask]
    w = 1 - u - v  # 第三个重心坐标
    
    # 获取对应面片的顶点（向量化操作）
    sampled_v0 = vertices[faces[sampled_face_indices, 0]]
    sampled_v1 = vertices[faces[sampled_face_indices, 1]]
    sampled_v2 = vertices[faces[sampled_face_indices, 2]]
    uniform_pc = (w[:, None] * sampled_v0 +
        u[:, None] * sampled_v1 +
        v[:, None] * sampled_v2)
    return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    results = np.zeros((sample_num, pc.shape[1]))
    results[0] = pc[np.random.choice(pc.shape[0])]
    distances = np.full(pc.shape[0], np.inf)
    for i in range(1, sample_num):
        for j in range(pc.shape[0]):
            dist = np.linalg.norm(pc[j] - results[:i], axis=1)
            distances[j] = np.min(dist)
        idx = np.argmax(distances)
        results[i] = pc[idx]

    return results

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [6]:
# task 3: metrics

from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0
CD = []
EMD = []

for i in tqdm.tqdm(range(5)):
    # uniform sampling
    _,_, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, final_sample_num)
    
    # FPS
    _,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

    # compute metrics
    cd = np.mean(np.min(np.linalg.norm(uniform_pc[:, None] - fps_pc[None], axis=2), axis=1)) + \
         np.mean(np.min(np.linalg.norm(fps_pc[:, None] - uniform_pc[None], axis=2), axis=1))
    CD.append(cd)
    emd = earthmover_distance(uniform_pc, [tuple(point) for point in fps_pc])
    EMD.append(emd)

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

print('CD mean: ', CD_mean)
print('CD var: ', CD_var)
print('EMD mean: ', EMD_mean)
print('EMD var: ', EMD_var)
# Visualization. For you to check your code
# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

100%|██████████| 5/5 [01:28<00:00, 17.60s/it]

CD mean:  2.7515918902005643
CD var:  0.0016560575957754043
EMD mean:  2.2914055266194904
EMD var:  0.010672555877896034
